In [ ]:
! pip install langchain
! pip install langchain-openai langchain-google-genai

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.0
    Uninstalling langchain-core-1.0.0:
      Successfully uninstalled langchain-core-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-openai 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
  Using cached langchain_core-1.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached langchain_core-1.0.0-py3-none-any.whl (467 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.7

In [ ]:
%%writefile /content/MCA-Insight-Engine/scripts/MCA-Dashboard.py
import pandas as pd
import glob as glob
import altair as alt
import streamlit as st
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from pyngrok import ngrok

# ---------------------------
# Streamlit App Title
# ---------------------------
st.title("MCA Insights Dashboard")

# ---------------------------
# Load Enriched Dataset
# ---------------------------
enriched_file = "/content/MCA-Insight-Engine/Data/Enriched/enriched_sample.csv"
if not os.path.exists(enriched_file):
    st.error(f"Master CSV not found: {enriched_file}")
    st.stop()

df = pd.read_csv(enriched_file)

# ---------------------------
# Master Dataset Filters
# ---------------------------
st.subheader("Master Dataset")
search_cin = st.text_input("Search by CIN")
search_name = st.text_input("Search by Company Name")

filtered_df = df.copy()
if search_cin:
    filtered_df = filtered_df[filtered_df["CIN"].str.contains(search_cin, case=False, na=False)]
if search_name:
    filtered_df = filtered_df[filtered_df["COMPANY_NAME"].str.contains(search_name, case=False, na=False)]

status_options = df["STATUS"].dropna().unique()
selected_status = st.multiselect("Filter by Status", status_options)
if selected_status:
    filtered_df = filtered_df[filtered_df["STATUS"].isin(selected_status)]

st.subheader(f"Showing {len(filtered_df)} records")
st.dataframe(filtered_df)

# ---------------------------
# Daily Summary
# ---------------------------
st.subheader("Summary of Changes by Status")
daily_summary = df.groupby("STATUS").size().reset_index(name="Count")
st.table(daily_summary)

# ---------------------------
# Daily Change Logs (Optional)
# ---------------------------
st.subheader("Daily Change Logs")
change_files = glob.glob("/content/MCA-Insight-Engine/Data/Change logs/*.csv")
if change_files:
    change_df = pd.concat([pd.read_csv(f) for f in change_files], ignore_index=True)

    change_type_options = change_df["Change_Type"].dropna().unique()
    selected_change_type = st.multiselect("Filter by Change Type", change_type_options)
    filtered_changes = change_df.copy()
    if selected_change_type:
        filtered_changes = filtered_changes[filtered_changes["Change_Type"].isin(selected_change_type)]

    st.write(f"Showing {len(filtered_changes)} change records")
    st.dataframe(filtered_changes)

    # Visualization of changes over time
    st.subheader("Change History Visualization")
    summary = filtered_changes.groupby(["Date","Change_Type"]).size().reset_index(name="Count")
    chart = alt.Chart(summary).mark_bar().encode(
        x='Date',
        y='Count',
        color='Change_Type'
    ).properties(width=700, height=400)
    st.altair_chart(chart, use_container_width=True)

    # Daily summary counts
    st.subheader("Daily Summary")
    total_new = len(filtered_changes[filtered_changes["Change_Type"]=="New Incorporation"])
    total_removed = len(filtered_changes[filtered_changes["Change_Type"]=="Deregistered"])
    total_updates = len(filtered_changes[filtered_changes["Change_Type"]=="Field Update"])

    st.write(f"New incorporations: {total_new}")
    st.write(f"Deregistered: {total_removed}")
    st.write(f"Updated records: {total_updates}")
else:
    st.info("No change log files found.")

# ---------------------------
# Gemini Chatbot Section
# ---------------------------

st.subheader("💬 Ask the MCA AI Assistant")

# Configure Gemini API Key
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY", "YOUR_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY  # ✅ make sure env var is set
genai.configure(api_key=GOOGLE_API_KEY)

# Create Chat Model (LangChain Wrapper)
chat_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # ✅ Correct model name
    temperature=0.3
)

# Output Parser
output_parser = StrOutputParser()

# Prompt Template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant specializing in Indian MCA data and company insights."),
    ("human", "Dataset summary:\n{context}\n\nUser question: {user_input}\nAnswer clearly and concisely."),
])

# Streamlit Input
user_input = st.text_input("Enter your question here:")

if user_input:
    try:
        # Prepare dataset context
        context_summary = f"The dataset contains {len(df)} companies. Here are some examples:\n"
        context_summary += "\n".join(
            df.head(5).apply(lambda row: f"{row['CIN']} - {row['COMPANY_NAME']} - {row['STATUS']}", axis=1)
        )

        # LangChain pipeline
        chain = prompt_template | chat_model | output_parser
        inputs = {"context": context_summary, "user_input": user_input}
        ai_response = chain.invoke(inputs)

        st.markdown(f"**AI Response:** {ai_response}")

    except Exception as e:
        st.error(f"Error while calling Gemini API: {e}")

Overwriting /content/MCA-Insight-Engine/scripts/MCA-Dashboard.py


In [ ]:

# Launch Streamlit app in background
from pyngrok import ngrok
ngrok.set_auth_token("34IHhQlx2iJMPJNW8Xn8rPJ24Qj_7SHRvpLCB578j8hfHs5Z3")
# Kill previous processes if any
!kill $(lsof -t -i:8501) || echo "No previous process"

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("🚀 Your Streamlit URL:", public_url)

# Run Streamlit app
!streamlit run /content/MCA-Insight-Engine/scripts/MCA-Dashboard.py --server.headless true --server.port 8501 &


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
No previous process
🚀 Your Streamlit URL: NgrokTunnel: "https://haydee-tippier-breathlessly.ngrok-free.dev" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.11.64.39:8501

